# Reconocimento de horas en relos analógico

Se utiliza el [dataset de relojes de Kaggles](https://www.kaggle.com/datasets/gpiosenka/time-image-datasetclassification) para entrenar un modelo de reconocimiento de horas en relojes analógicos.

In [1]:
import pandas as pd

## Decompresión de imágenes

Descomprimir dataset para trabajar con ello. Estos ficheros no se subiran al github

In [3]:
import zipfile

local_zip = './tmp/archive.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./Assets/dataset')
zip_ref.close()

KeyboardInterrupt: 

## Inicialización de datos

## Lectura fichero csv

In [4]:
csv_data = pd.read_csv('Assets/dataset/clocks.csv')

In [9]:
hours_array = csv_data['labels'].unique()

In [5]:
data_test = csv_data[csv_data['data set'] == "test"]